# Test loading up checkpoint

### Imports

In [1]:
from __future__ import annotations

import os.path as osp
import torch
import torch.nn as nn
import wandb
from tqdm.auto import trange

import lqsvg.experiment.utils as utils
from lqsvg.envs import lqr
from lqsvg.experiment.policy import make_worker
from lqsvg.experiment.models import MonteCarloSVG
from lqsvg.experiment.models import AnalyticSVG

In [2]:
CONFIG = dict(n_state=8, n_ctrl=8, horizon=1000, svg_samples=10, monte_carlo_svg=32, iterations=200)

run = wandb.init(
    job_type="eval",
    config=CONFIG,
    project="LQG-SVG",
    entity="angelovtt",
    reinit=False,
    tags=["eval", utils.calver()],
    name="Eval Grads",
    mode="online",
    allow_val_change=False,
    save_code=True,
)
hparams = run.config
artifact = run.use_artifact(f"angelovtt/LQG-SVG/svg_inf-lqg{hparams.n_state}.{hparams.n_ctrl}.{hparams.horizon}:latest", type="model")
artifact_dir = artifact.download()

wandb: Currently logged in as: angelovtt (use `wandb login --relogin` to force relogin)


In [3]:
print("Artifact dir:", artifact_dir)
worker = make_worker(dict(n_state=hparams.n_state, n_ctrl=hparams.n_ctrl, horizon=hparams.horizon, num_envs=1))
key_mismatch = worker.env.module.load_state_dict(torch.load(osp.join(artifact_dir, "mdp.pt")))
assert not any(key_mismatch), key_mismatch

Artifact dir: ./artifacts/svg_inf-lqg8.8.1000:v6


/Users/angelolovatto/Library/Caches/pypoetry/virtualenvs/lqsvg-kEsPsxvA-py3.8/lib/python3.8/site-packages/torch/tensor.py:758: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ../c10/core/TensorImpl.h:930.)
  return super(Tensor, self).refine_names(names)


In [4]:
mdp = worker.env.module
module = worker.get_policy().module

In [5]:
def restore_iteration(module: nn.Module, iteration: int):
    mismatch = module.load_state_dict(torch.load(osp.join(artifact_dir, f"module-iter={iteration}.pt")))
    assert not any(mismatch), mismatch

In [6]:
def generate_svg_samples() -> list[lqr.Linear]:
    svg_estimator = MonteCarloSVG(module.actor, module.model)
    return [svg_estimator(hparams.monte_carlo_svg)[1] for _ in range(hparams.svg_samples)]

In [7]:
def log_gradient_error(svg_samples: list[lqr.Linear]):
    solver = AnalyticSVG(module.actor, mdp)
    true_value, true_svg = solver()

    cossims = [utils.linear_feedback_cossim(g, true_svg) for g in svg_samples]
    run.log({"cossim with true grad": torch.stack(cossims).mean().item()}, commit=False)

In [8]:
def log_empirical_variance(svg_samples: list[lqr.Linear]):
    cossims = []
    for i, gi in enumerate(svg_samples):
        for gj in svg_samples[i+1:]:
            cossims += [utils.linear_feedback_cossim(gi, gj)]
            
    run.log({"avg. pairwise cossim": torch.stack(cossims).mean().item()}, commit=False)

In [9]:
def log_iteration(iteration: int):
    restore_iteration(module, iteration)
    svg_samples = generate_svg_samples()
    log_gradient_error(svg_samples)
    log_empirical_variance(svg_samples)
    run.log({"iteration": iteration})

In [10]:
for iteration in trange(0, hparams.iterations + 1, 10, desc="Evaluating", unit="ckpt"):
    log_iteration(iteration)
    
run.finish()

Evaluating:   0%|          | 0/21 [00:00<?, ?ckpt/s]

cossim with true grad,-0.05521
avg. pairwise cossim,0.59789
iteration,200
_runtime,987
_timestamp,1616166872
_step,20


cossim with true grad,▂█▇▆▅▄▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁
avg. pairwise cossim,▇█▇▇▆▅▅▅▄▄▄▃▃▃▃▂▂▁▁▁▁
iteration,▁▁▂▂▂▃▃▃▄▄▅▅▅▆▆▆▇▇▇██
_runtime,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▆▇▇▇██
_timestamp,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▆▇▇▇██
_step,▁▁▂▂▂▃▃▃▄▄▅▅▅▆▆▆▇▇▇██
